## Requirements

In [ ]:
!pip install git+https://github.com/huggingface/diffusers.git
!pip install accelerate
!pip install datasets
!pip install bitsandbytes

In [ ]:
!git clone https://huggingface.co/datasets/GP-Alternativweltgeschichten/Satbilder

In [ ]:
import torch
from diffusers import StableDiffusionInpaintPipeline
from PIL import Image
import matplotlib.pyplot as plt

## Training the model

#### In terminal (Pycharm tends to crash after a while when using Jupyter Notebook):
accelerate launch diffusers/examples/text_to_image/train_text_to_image.py --pretrained_model_name_or_path=stabilityai/stable-diffusion-2-1 --dataset_name=SatBilder --use_ema --use_8bit_adam --resolution=768 --center_crop --random_flip --train_batch_size=1 --gradient_accumulation_steps=4 --gradient_checkpointing --mixed_precision="fp16" --max_train_steps=10000 --learning_rate=1e-05 --max_grad_norm=1 --lr_scheduler="constant" --lr_warmup_steps=0 --output_dir="Olpe_Model" --checkpointing_steps=5000

#### In Jupyter:

In [ ]:
!accelerate launch diffusers/examples/text_to_image/train_text_to_image.py \
  --pretrained_model_name_or_path=stabilityai/stable-diffusion-2-1 \
  --dataset_name=SatBilder \
  --use_ema \
  --use_8bit_adam \
  --resolution=768 --center_crop --random_flip \
  --train_batch_size=8 \
  --gradient_accumulation_steps=2 \
  --gradient_checkpointing \
  --mixed_precision="fp16" \
  --max_train_steps=10000 \
  --learning_rate=1e-05 \
  --max_grad_norm=1 \
  --lr_scheduler="constant" --lr_warmup_steps=0 \
  --output_dir="Olpe_Model"

## Testing the model

In [ ]:
pipe = StableDiffusionInpaintPipeline.from_pretrained('./Olpe_Model', torch_dtype=torch.float16)
pipe = pipe.to("cuda")

In [ ]:
initial_image = Image.open("./Test_Images/input.png")
mask_image = Image.open("./Test_Images/mask.png")

plt.subplot(1, 2, 1)
plt.imshow(initial_image)
plt.axis('off')
plt.title("Initial Image")

plt.subplot(1, 2, 2)
plt.imshow(mask_image)
plt.axis('off')
plt.title("Mask Image")

In [ ]:
prompt = "Aerial view of the city Olpe: A blue river."
img = pipe(prompt, image=initial_image, mask_image=mask_image, strength=0.9, num_inference_steps=200).images[0]
img

In [ ]:
prompt = "Aerial view of the city Olpe: A church."
img = pipe(prompt, image=initial_image, mask_image=mask_image, strength=0.9, num_inference_steps=200).images[0]
img